In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
    

In [2]:
df = pd.read_csv('3_Merged_Data/Merged_Suicidal_WHR_2023-07-23-22-20-29.csv')

df.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Regional indicator,Crude
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258,South Asia,4.61
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237,South Asia,4.39
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275,South Asia,4.27
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267,South Asia,4.12
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268,South Asia,4.01


In [6]:
df.describe()

,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Crude
count,1735.000000,1735.000000,1729.000000,1726.000000,1735.000000,1707.000000,1678.000000,1637.000000,1719.000000,1723.000000,1735.000000
mean,2012.896830,5.468263,9.361573,0.813615,63.398686,0.742696,-0.002415,0.750674,0.711706,0.266724,10.307424
std,4.003785,1.125754,1.146901,0.118186,7.457788,0.140762,0.164721,0.186078,0.107268,0.081557,7.889194
min,2005.000000,2.375000,6.635000,0.291000,32.300000,0.258000,-0.335000,0.035000,0.322000,0.095000,0.560000
25%,2010.000000,4.635500,8.453000,0.749000,58.750000,0.646000,-0.118000,0.696000,0.628000,0.206000,5.230000
50%,2013.000000,5.374000,9.461000,0.838000,65.200000,0.760000,-0.030000,0.805000,0.725000,0.256000,8.040000
75%,2016.000000,6.294500,10.331000,0.908000,68.500000,0.855500,0.088000,0.875000,0.803000,0.316000,12.875000
max,2019.000000,8.019000,11.648000,0.987000,77.100000,0.985000,0.698000,0.983000,0.944000,0.591000,87.000000


In [7]:
cols = ['Life Ladder','Log GDP per capita', 'Social support', 'Healthy life expectancy at birth',
        'Freedom to make life choices', 'Generosity','Perceptions of corruption', 'Crude']

df_new = df[cols]

In [8]:
df_new.head()

,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Crude
0,3.724,7.370,0.451,50.80,0.718,0.168,0.882,4.61
1,4.402,7.540,0.552,51.20,0.679,0.190,0.850,4.39
2,4.758,7.647,0.539,51.60,0.600,0.121,0.707,4.27
3,3.832,7.620,0.521,51.92,0.496,0.162,0.731,4.12
4,3.783,7.705,0.521,52.24,0.531,0.236,0.776,4.01


In [17]:
(round(df_new.isna().sum()*100/len(df_new),2))

Life Ladder                         0.0
Log GDP per capita                  0.0
Social support                      0.0
Healthy life expectancy at birth    0.0
Freedom to make life choices        0.0
Generosity                          0.0
Perceptions of corruption           0.0
Crude                               0.0
dtype: float64

In [16]:
df_new.dropna(inplace=True)

/tmp/ipykernel_12287/420984286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.dropna(inplace=True)


In [18]:
X = df_new.drop('Life Ladder',axis=1)
y = df_new['Life Ladder']

In [19]:
print("Shape of X ",X.shape)
print("Shape of y", y.shape)

Shape of X  (1579, 7)
Shape of y (1579,)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=42)

In [25]:
#Creating a model object and fiting the data
model = LinearRegression()
model.fit(X_train,y_train)

y_test_pred =model.predict(X_test) #predicting the Test data

r2_score = model.score(X_test,y_test)
print("Accuracy score -- ",r2_score*100,'%')

Accuracy score --  76.19297962283973 %


In [26]:
l_reg = pd.DataFrame({'Features': X.columns,
                      'Importance_Score': model.coef_[0]
                     }).sort_values(by='Importance_Score',ascending=False)

In [27]:
l_reg

,Features,Importance_Score
0,Log GDP per capita,0.408962
1,Social support,0.408962
2,Healthy life expectancy at birth,0.408962
3,Freedom to make life choices,0.408962
4,Generosity,0.408962
5,Perceptions of corruption,0.408962
6,Crude,0.408962
